In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df= pd.read_parquet('/workspaces/1mlopszoomcamp/yellow_tripdata_2023-01.parquet')
df.shape[1]

19

In [3]:
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df['duration']=df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration']= df.duration.apply(lambda td: td.total_seconds()/60)

df.duration.std()

42.594351241920904

In [4]:
frac_left = ((df.duration >=1)&(df.duration<=60)).mean()
print(frac_left *100)

98.1220282212598


In [5]:
df = df[(df.duration >= 1)&(df.duration <= 60)]

categorical = ['PULocationID','DOLocationID']

df[categorical]=df[categorical].astype(str)

train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)

X_train.shape[1]

515

In [6]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train,y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared= False)

7.6492610279057605

In [7]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
        
    
#Add new column
    df['duration']=df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration']= df.duration.apply(lambda td: td.total_seconds()/60)

    df = df[(df.duration >= 1)&(df.duration <= 60)]

    categorical = ['PULocationID','DOLocationID']
    df[categorical]=df[categorical].astype(str)
    
    return df

In [ ]:
df_val= read_dataframe('/workspaces/1mlopszoomcamp/yellow_tripdata_2023-02.parquet')

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_val = df_val[target].values

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared= False)